# Statistics
### Martin Valapka - xvalapm00

# Hypothesis 1: The probability of dying in an accident on first‑class roads is the same as on third‑class roads.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import contextily
import sklearn.cluster
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency, mannwhitneyu

### Load accident data

In [2]:
df_accidents = pd.read_pickle("accidents.pkl.gz")
df_accidents

,p1,p2a,p2b,p4a,p4b,p4c,p5a,p6,p7,p8,...,p27,p28,p34,p35,p36,p37,p38,p39,date,region
0,10623000002,01.01.2023,1730,1,7,12,2,5,0,0,...,0,1,1,0,1,16.0,7532.0,NaN,2023-01-01,STC
1,60223000039,10.01.2023,916,6,2,10,1,3,0,3,...,0,5,1,10,6,NaN,NaN,6.0,2023-01-10,JHM
2,23000455,15.01.2023,1930,0,11,20,1,1,4,0,...,10,1,2,0,6,NaN,NaN,NaN,2023-01-15,PHA
3,70623000019,10.01.2023,815,7,6,18,2,5,0,0,...,0,1,1,0,2,460.0,1275.0,NaN,2023-01-10,MSK
4,161723000024,11.01.2023,1730,16,17,17,2,5,0,0,...,0,1,1,0,6,NaN,NaN,NaN,2023-01-11,VYS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63317,161025000500,22.09.2025,1642,16,10,11,2,9,0,0,...,0,1,1,0,2,152.0,8430.0,NaN,2025-09-22,VYS
63318,161725000414,23.09.2025,730,16,17,12,2,3,0,1,...,0,3,1,0,2,347.0,2037.0,NaN,2025-09-23,VYS
63319,161625000400,23.09.2025,2560,16,16,12,1,2,0,0,...,10,1,2,0,6,NaN,NaN,NaN,2025-09-23,VYS
63320,140825000658,03.10.2025,145,14,8,19,2,5,0,0,...,0,1,1,0,3,43610.0,57.0,NaN,2025-10-03,OLK


### Create columns, get required rows

In [3]:
df_accidents["p36_num"] = pd.to_numeric(df_accidents["p36"], errors="coerce")
df_accidents["p9_num"] = pd.to_numeric(df_accidents["p9"], errors="coerce")

first_class = df_accidents[df_accidents["p36_num"] == 1]
third_class = df_accidents[df_accidents["p36_num"] == 3]
highway = df_accidents[df_accidents["p36_num"] == 0]

injury_accident = df_accidents[df_accidents["p9_num"] == 1]

print(f"First-class roads accidents: {len(first_class)}")
print(f"Third-class roads accidents: {len(third_class)}")
print(f"Accidents with injuries: {len(injury_accident)}")

alpha = 0.05

First-class roads accidents: 32123
Third-class roads accidents: 34386
Accidents with injuries: 58784


### Calculation, results and conclusion

In [4]:
def chi2_compare(df, group_col, value_col, group_a, group_b, index_names=None, column_names=None, focus_label=None, alpha=0.05, title=None):
    """
    @brief Perform Chi-square test comparing two groups in a DataFrame
    @param df: DataFrame containing the data
    @param group_col: Column name for grouping variable
    @param value_col: Column name for outcome variable
    @param group_a: Value of group_col for the first group
    @param group_b: Value of group_col for the second group
    @param index_names: Optional list of names for the index of the contingency table
    @param column_names: Optional list of names for the columns of the contingency table
    @param focus_label: Optional label for the group to focus on in the conclusion
    @param alpha: Significance level for the test
    @param title: Optional title for the output
    """
    
    df_sub = df[df[group_col].isin([group_a, group_b])].copy()
    ct = pd.crosstab(df_sub[group_col], df_sub[value_col])
    if index_names is not None:
        ct.index = index_names
    if column_names is not None:
        ct.columns = column_names
    
    if title is not None:
        print("\n" + "="*50)
        print(title)
        print("="*50)
    
    print(ct)
    
    chi2, p, dof, expected = stats.chi2_contingency(ct)
    
    if title is None:
        print("\n--- Chi-square test results (1st vs 3rd class) ---")
    else:
        print("\n--- Chi-square test results ---")
    print(f"χ² statistic: {chi2:.4f}")
    print(f"p-value: {p:.4e}")
    
    expected_df = pd.DataFrame(expected, index=ct.index, columns=ct.columns)
    print("\nExpected frequencies:")
    print(expected_df.round(2))
    
    print(f"\n--- CONCLUSION ---")
    focus = focus_label if focus_label is not None else (index_names[0] if index_names is not None else group_a)
    
    if p < alpha:
        print(f"p-value ({p:.4e}) < {alpha}: REJECT null hypothesis")
        print("The probability of health consequences DIFFERS between {0} and {1}".format(index_names[0] if index_names is not None else group_a, index_names[1] if index_names is not None else group_b))
        actual_focus_inj = ct.loc[focus, "Injury/Death"]
        expected_focus_inj = expected_df.loc[focus, "Injury/Death"]
        
        if actual_focus_inj > expected_focus_inj:
            print(f"{focus}: MORE injuries/deaths than expected ({actual_focus_inj:.0f} vs {expected_focus_inj:.0f})")
            if title is not None and focus == "1. class":
                print(f"CONCLUSION: Accidents on 1st-class roads result in injuries MORE OFTEN than on highways.")
        else:
            print(f"{focus}: FEWER injuries/deaths than expected ({actual_focus_inj:.0f} vs {expected_focus_inj:.0f})")
            if title is not None and focus == "1. class":
                print(f"CONCLUSION: Accidents on 1st-class roads result in injuries LESS OFTEN than on highways.")
    else:
        print(f"p-value ({p:.4e}) >= {alpha}: FAIL TO REJECT null hypothesis")
        print("No significant difference in health consequences between {0} and {1}.".format(index_names[0] if index_names is not None else group_a, index_names[1] if index_names is not None else group_b))

### Run analysis

In [5]:
chi2_compare(df_accidents, 'p36_num', 'p9_num', 1, 3, index_names=["1. class", "3. class"], column_names=["Injury/Death", "Property damage"], focus_label="1. class", alpha=alpha, title=None)
chi2_compare(df_accidents, 'p36_num', 'p9_num', 0, 1, index_names=["Highways", "1. class"], column_names=["Injury/Death", "Property damage"], focus_label="1. class", alpha=alpha, title="COMPARISON: 1st Class Roads vs Highways")

          Injury/Death  Property damage
1. class         10894            21229
3. class         10747            23639

--- Chi-square test results (1st vs 3rd class) ---
χ² statistic: 53.3884
p-value: 2.7371e-13

Expected frequencies:
          Injury/Death  Property damage
1. class      10452.33         21670.67
3. class      11188.67         23197.33

--- CONCLUSION ---
p-value (2.7371e-13) < 0.05: REJECT null hypothesis
The probability of health consequences DIFFERS between 1. class and 3. class
1. class: MORE injuries/deaths than expected (10894 vs 10452)

COMPARISON: 1st Class Roads vs Highways
          Injury/Death  Property damage
Highways          1940             9910
1. class         10894            21229

--- Chi-square test results ---
χ² statistic: 1288.0430
p-value: 4.4840e-282

Expected frequencies:
          Injury/Death  Property damage
Highways       3458.55          8391.45
1. class       9375.45         22747.55

--- CONCLUSION ---
p-value (4.4840e-282) < 0.05: 

# Hypothesis 2: Vehicle damage comparison between brands

## Hypothesis
The damage for brand X is lower than for brand Y in accidents and this difference is statistically significant.

**Test selection**: Mann-Whitney U test
- Samples are independent
- Non-Normal Distribution
- Unlike t-test, Mann-Whitney U test uses ranks    

**How the test works**
The test ranks all damage values from both brands combined and calculates the probability that one brand's damage values are systematically higher or lower than the other's.

**What the mannwhitneyu(a, b) function does step by step:**
1. **Combine the data**: Takes all damage values from brand A and brand B and merges them into one list
2. **Rank all values**: Sorts all damage values from smallest to largest and assigns each a rank (1 = smallest, N = largest)
3. **Sum the ranks for each brand**: 
   - Adds up all the ranks assigned to brand A values → Sum_A
   - Adds up all the ranks assigned to brand B values → Sum_B
4. **Calculate the U statistic**: Uses the formula U = Sum_A - (nA × (nA + 1) / 2) where nA is the sample size of brand A
5. **Compare to the null distribution**: Determines the probability (p-value) that this U value would occur if there was no real difference between the brands

### Load vehicle data

In [6]:
df_vehicles = pd.read_pickle("vehicles.pkl.gz")
df_vehicles

,p1,id_vozidla,p44,p45a,p45b,p45d,p45f,p47,p48a,p48b,...,p50a,p50b,p51,p52,p53,p55a,p55b,p56,p57,p58
0,22000026,1,3,39.0,12.0,2.0,2.0,03,3.0,0.0,...,4.0,0.0,1.0,5,100,9.0,NaN,XX,NaN,NaN
1,23000010,1,5,29.0,24.0,2.0,2.0,09,1.0,0.0,...,1.0,0.0,1.0,5,800,2.0,1.0,xx,1.0,1.0
2,23000011,1,3,26.0,32.0,1.0,0.0,00,3.0,0.0,...,4.0,0.0,1.0,5,0,9.0,NaN,XX,NaN,NaN
3,23000011,2,3,32.0,16.0,1.0,2.0,05,3.0,0.0,...,1.0,0.0,1.0,5,100,2.0,3.0,25,1.0,1.0
4,23000012,1,3,39.0,15.0,2.0,2.0,22,3.0,0.0,...,0.0,0.0,1.0,5,600,2.0,3.0,01,5.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105619,194025000061,1,3,17.0,20.0,1.0,1.0,13,1.0,0.0,...,1.0,0.0,1.0,1,0,2.0,0.0,XX,1.0,2.0
105620,194025000061,2,3,39.0,20.0,1.0,1.0,17,1.0,0.0,...,1.0,0.0,1.0,1,10,2.0,0.0,XX,1.0,1.0
105621,194025000062,1,3,17.0,11.0,2.0,0.0,06,1.0,0.0,...,1.0,0.0,1.0,3,300,2.0,0.0,XX,1.0,1.0
105622,194025000064,1,3,39.0,13.0,2.0,2.0,96,1.0,0.0,...,1.0,0.0,1.0,3,50,2.0,0.0,XX,1.0,1.0


### Create columns, get required rows

In [7]:
df_vehicles["p45a_num"] = pd.to_numeric(df_vehicles["p45a"], errors="coerce")
df_vehicles["p53_num"] = pd.to_numeric(df_vehicles["p53"], errors="coerce")

alpha = 0.05

### Calculation, results and conclusion

In [8]:
def compare_brands(df, code_a, code_b, name_a=None, name_b=None, alpha=0.05):
    """
    @brief Compare damage costs between two vehicle brands using Mann-Whitney U test
    @param df: DataFrame containing vehicle data
    @param code_a: Code for the first vehicle brand
    @param code_b: Code for the second vehicle brand
    @param name_a: Optional name for the first vehicle brand
    @param name_b: Optional name for the second vehicle brand
    @param alpha: Significance level for the test
    """

    name_a = name_a or f"Brand {code_a}"
    name_b = name_b or f"Brand {code_b}"

    a = df[df["p45a_num"] == code_a]["p53_num"].dropna()
    b = df[df["p45a_num"] == code_b]["p53_num"].dropna()

    print("\n" + "-"*60)
    print(f"COMPARISON: {name_a} vs {name_b}")
    print("-"*60)
    print(f"{name_a}: n={len(a)}, mean={a.mean():.2f}, std={a.std():.2f}, median={a.median():.2f}")
    print(f"{name_b}: n={len(b)}, mean={b.mean():.2f}, std={b.std():.2f}, median={b.median():.2f}")

    stat, p_value = mannwhitneyu(a, b)
    print(f"\nMann-Whitney U test results:")
    print(f"Test statistic: {stat:.4f}")
    print(f"p-value: {p_value:.4e}")

    print(f"\n--- CONCLUSION ({name_a} vs {name_b}) ---")
    if p_value < alpha:
        print(f"p-value ({p_value:.4e}) < {alpha}: Result is STATISTICALLY SIGNIFICANT")
        if a.median() < b.median():
            print(f"{name_a} damage is LOWER...")
        elif a.median() > b.median():
            print(f"{name_a} damage is HIGHER...")
        else:
            print(f"{name_a} and {name_b} have the SAME median ({a.median():.2f})")
            print(f"BUT: {name_b} has HIGHER variability (std: {b.std():.2f} vs {a.std():.2f})")
            print(f"This indicates more extreme damage values in {name_b} accidents")
    else:
        print(f"p-value ({p_value:.4e}) >= {alpha}: Result is NOT statistically significant")
        print("Cannot determine if damage difference is significant")

    return p_value

compare_brands(df_vehicles, 39, 47, name_a="Škoda", name_b="Volkswagen", alpha=alpha)
compare_brands(df_vehicles, 26, 47, name_a="Mercedes", name_b="Volkswagen", alpha=alpha)


------------------------------------------------------------
COMPARISON: Škoda vs Volkswagen
------------------------------------------------------------
Škoda: n=93551, mean=494.11, std=705.36, median=300.00
Volkswagen: n=37166, mean=485.37, std=733.88, median=300.00

Mann-Whitney U test results:
Test statistic: 1772562872.0000
p-value: 2.6706e-08

--- CONCLUSION (Škoda vs Volkswagen) ---
p-value (2.6706e-08) < 0.05: Result is STATISTICALLY SIGNIFICANT
Škoda and Volkswagen have the SAME median (300.00)
BUT: Volkswagen has HIGHER variability (std: 733.88 vs 705.36)
This indicates more extreme damage values in Volkswagen accidents

------------------------------------------------------------
COMPARISON: Mercedes vs Volkswagen
------------------------------------------------------------
Mercedes: n=16282, mean=764.65, std=1847.21, median=300.00
Volkswagen: n=37166, mean=485.37, std=733.88, median=300.00

Mann-Whitney U test results:
Test statistic: 301739319.0000
p-value: 6.1242e-01

--

np.float64(0.6124246926259953)

### Run analysis

In [9]:
compare_brands(df_vehicles, 39, 47, name_a="Škoda", name_b="Volkswagen", alpha=alpha)
compare_brands(df_vehicles, 26, 47, name_a="Mercedes", name_b="Volkswagen", alpha=alpha)


------------------------------------------------------------
COMPARISON: Škoda vs Volkswagen
------------------------------------------------------------
Škoda: n=93551, mean=494.11, std=705.36, median=300.00
Volkswagen: n=37166, mean=485.37, std=733.88, median=300.00

Mann-Whitney U test results:
Test statistic: 1772562872.0000
p-value: 2.6706e-08

--- CONCLUSION (Škoda vs Volkswagen) ---
p-value (2.6706e-08) < 0.05: Result is STATISTICALLY SIGNIFICANT
Škoda and Volkswagen have the SAME median (300.00)
BUT: Volkswagen has HIGHER variability (std: 733.88 vs 705.36)
This indicates more extreme damage values in Volkswagen accidents

------------------------------------------------------------
COMPARISON: Mercedes vs Volkswagen
------------------------------------------------------------
Mercedes: n=16282, mean=764.65, std=1847.21, median=300.00
Volkswagen: n=37166, mean=485.37, std=733.88, median=300.00

Mann-Whitney U test results:
Test statistic: 301739319.0000
p-value: 6.1242e-01

--

np.float64(0.6124246926259953)